In [ ]:
import requests
import json
from typing import List, Dict, Any
import statistics
from pathlib import Path
from collections import Counter
from scipy.stats import pearsonr

class MoodleReadabilityAPI:
    def __init__(self, base_url: str, token: str):
        self.base_url = base_url
        self.token = token
        self.endpoint = f"{self.base_url}/webservice/rest/server.php"

    def calculate_readability(self, texts: List[str], urls: List[str]) -> List[Dict[str, Any]]:
        if len(texts) != len(urls):
            raise ValueError("The number of texts and URLs must match")

        results = []
        for text, url in zip(texts, urls):
            params = {
                'wstoken': self.token,
                'wsfunction': 'block_readabilityscore_process_text',
                'moodlewsrestformat': 'json',
                'selectedtext': text,
                'pageurl': url
            }

            try:
                response = requests.post(self.endpoint, data=params)
                response.raise_for_status()
                result = response.json()
                results.append(result)
            except requests.exceptions.RequestException as e:
                raise

        return results

def read_corpus(base_dir: str) -> List[Dict[str, Any]]:
    corpus_dir = Path(base_dir) / "Texts-SeparatedByReadingLevel"
    if not corpus_dir.is_dir():
        raise FileNotFoundError(f"Corpus directory not found: {corpus_dir}")

    corpus = []
    level_dirs = {'Ele-Txt': 'elementary', 'Int-Txt': 'intermediate', 'Adv-Txt': 'advanced'}
    
    for dir_name, level in level_dirs.items():
        level_dir = corpus_dir / dir_name
        if not level_dir.is_dir():
            continue
        
        for file_path in level_dir.glob('*.txt'):
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read().strip()
                corpus.append({
                    'text': text,
                    'level': level,
                    'file': file_path.name
                })
    
    return corpus

def classify_score(score: float) -> str:
    if score <= 9:
        return 'elementary'
    elif score <= 13:
        return 'intermediate'
    else:
        return 'advanced'

def calculate_confidence(score: float) -> float:
    if score <= 8.5 or score >= 13.5:
        return 1.0
    elif 8.5 < score <= 9.5 or 12.5 < score <= 13.5:
        return 0.7
    else:
        return 0.5

def validate_accuracy(api: MoodleReadabilityAPI, corpus: List[Dict[str, Any]]) -> Dict[str, Any]:
    texts = [item['text'] for item in corpus]
    urls = [f"http://example.com/{item['file']}" for item in corpus]
    results = api.calculate_readability(texts, urls)

    plugin_scores = [result['readabilityscore'] for result in results]
    corpus_levels = [item['level'] for item in corpus]

    level_map = {'elementary': 1, 'intermediate': 2, 'advanced': 3}
    mapped_levels = [level_map[classify_score(score)] for score in plugin_scores]
    
    correlation, _ = pearsonr(plugin_scores, mapped_levels)

    correct_classifications = sum(1 for result, item in zip(results, corpus) if classify_score(result['readabilityscore']) == item['level'])
    accuracy = correct_classifications / len(corpus)

    level_accuracy = {}
    level_confusion = {level: Counter() for level in ['elementary', 'intermediate', 'advanced']}
    detailed_results = []

    for item, result in zip(corpus, results):
        score = result['readabilityscore']
        predicted_level = classify_score(score)
        confidence = calculate_confidence(score)
        
        level_confusion[item['level']][predicted_level] += 1
        
        detailed_results.append({
            'file': item['file'],
            'actual_level': item['level'],
            'predicted_level': predicted_level,
            'gunning_fog_index': score,
            'confidence': confidence,
            'continuous_scale': score
        })

    for level in ['elementary', 'intermediate', 'advanced']:
        level_items = [item for item in corpus if item['level'] == level]
        correct = sum(1 for result in detailed_results if result['actual_level'] == level and result['actual_level'] == result['predicted_level'])
        level_accuracy[level] = correct / len(level_items) if level_items else 0

    weighted_correct = sum(1 * result['confidence'] for result in detailed_results if result['actual_level'] == result['predicted_level'])
    total_weight = sum(result['confidence'] for result in detailed_results)
    weighted_accuracy = weighted_correct / total_weight if total_weight > 0 else 0

    return {
        'correlation': correlation,
        'overall_accuracy': accuracy,
        'weighted_accuracy': weighted_accuracy,
        'level_accuracy': level_accuracy,
        'level_confusion': level_confusion,
        'detailed_results': detailed_results
    }

if __name__ == "__main__":
    api = MoodleReadabilityAPI("http://192.168.178.108:8000", "de39a2339ba3cfe58a55e0711ca716d6")
    base_dir = "/home/anrichp/Documents/Development/OneStopEnglishCorpus"
    
    try:
        corpus = read_corpus(base_dir)
        accuracy_results = validate_accuracy(api, corpus)
        
        print("\nAccuracy Validation Results:")
        print(f"Overall Accuracy: {accuracy_results['overall_accuracy']:.2%}")
        print(f"Weighted Accuracy: {accuracy_results['weighted_accuracy']:.2%}")
        print(f"Correlation: {accuracy_results['correlation']:.2f}")
        
        print("\nAccuracy by Level:")
        for level, acc in accuracy_results['level_accuracy'].items():
            print(f"  {level.capitalize()}: {acc:.2%}")

        print("\nConfusion Matrix:")
        for actual_level, predictions in accuracy_results['level_confusion'].items():
            print(f"  {actual_level.capitalize()}:")
            for predicted_level, count in predictions.items():
                print(f"    Predicted as {predicted_level}: {count}")

        print("\nDetailed Results:")
        for result in accuracy_results['detailed_results']:
            print(f"File: {result['file']}")
            print(f"  Actual Level: {result['actual_level']}")
            print(f"  Predicted Level: {result['predicted_level']}")
            print(f"  Gunning Fog Index: {result['gunning_fog_index']:.2f}")
            print(f"  Confidence: {result['confidence']:.2f}")
            print(f"  Continuous Scale: {result['continuous_scale']:.2f}")
            print()

    except FileNotFoundError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")